In [ ]:
if True:
    %pip install git+https://github.com/arendjan/tmx-pico-aio.git@sensors
    %pip install nest_asyncio

Hardware setup:

![button](images/load_cell_adc_bb.png)

Pico connections:

|Pin # | type  | connection |
|-------|------|------------|
| 18   | GND   |   Excitation -         |
| 31   | GP26  | Analog input, ADC0      |
| 32   | GP27  | Analog input, ADC1      |
| 36   | 3V3   |   Excitation +         |



In [ ]:
import asyncio
import sys
import time
from tmx_pico_aio import tmx_pico_aio
import nest_asyncio
import struct
nest_asyncio.apply() # for jupyter notebook asyncio stuff

loop = asyncio.get_event_loop()

board = tmx_pico_aio.TmxPicoAio(loop=loop)

Add your callback, like printing or adding it to a list

In [ ]:
lastA = 0
async def the_callback(data): # only will be called when there is a change
    global lastA
    lastA = data[2]
    print("a:", data[2])
async def the_callback2(data): # only will be called when there is a change
    print("b:", data[2], data[2]-lastA)

Add the sensor to the system

In [ ]:
async def addSensor(board):
    await asyncio.sleep(1)
    adc = 0 # pin GP26
    adc2 = 1 # pin GP27
    await board.set_pin_mode_analog_input(adc, differential=0, callback=the_callback)
    await board.set_pin_mode_analog_input(adc2, differential=0, callback=the_callback2)

loop.run_until_complete(addSensor(board))



Run for some time

In [ ]:
async def run_till_end(board):
    # when to stop?
    await asyncio.sleep(10)
    await board.shutdown() # Otherwise next start will give you errors and you need to restart the kernel
    await asyncio.sleep(1)
    print("done")

loop.run_until_complete(run_till_end(board))